In [1]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np
import os as os
import json

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers, callbacks, models
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Flatten
from datetime import datetime
from keras_self_attention import SeqSelfAttention

from keras import regularizers

import keras.backend as K

import collections

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        if int(nid) == 1: story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            substory = [[str(i)+":"]+x for i,x in enumerate(story) if x]
            data.append((substory, q, a))
            story.append('')
        else: story.append(tokenize(line))
    return data


def get_stories(f):
    data = parse_stories(f.readlines())
    return [(story, q, answer) for story, q, answer in data]


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []; Xq = []; Y = []
    for story, query, answer in data:
        x = [[word_idx[w] for w in s] for s in story]
        xq = [word_idx[w] for w in query]
        y = [word_idx[answer]]
        X.append(x); Xq.append(xq); Y.append(y)
    return ([pad_sequences(x, maxlen=story_maxlen) for x in X],
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

def do_flatten(el): 
    return isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes))
def flatten(l):
    for el in l:
        if do_flatten(el): yield from flatten(el)
        else: yield el
            
def stack_inputs(inputs):
    for i,it in enumerate(inputs):
        inputs[i] = np.concatenate([it, 
                           np.zeros((story_maxsents-it.shape[0],story_maxlen), 'int')])
    return np.stack(inputs)



In [3]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 128
EPOCHS = 40
MODEL_NAME = "LSTM_SelfAttention"
# Regularization parameter
LAMBDA = 0.02
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

try:
    path = get_file('babi-tasks-v1-2.tar.gz',
                    origin='https://s3.amazonaws.com/text-datasets/'
                           'babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2'
          '.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise




RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [33]:
# Default QA1 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
with tarfile.open(path) as tar:
    train_stories = get_stories(tar.extractfile(challenge.format('train')))
    test_stories = get_stories(tar.extractfile(challenge.format('test')))
np.random.shuffle(train_stories)
np.random.shuffle(test_stories)

stories = train_stories + test_stories

story_maxlen = max((len(s) for x, _, _ in stories for s in x))
story_maxsents = max((len(x) for x, _, _ in stories))
query_maxlen = max(len(x) for _, x, _ in stories)

vocab = sorted(set(flatten(stories)))
vocab.insert(0, '<PAD>')
vocab_size = len(vocab)

word_idx = dict((c, i) for i, c in enumerate(vocab))

inputs_train, queries_train, answers_train = vectorize_stories(train_stories, 
     word_idx, story_maxlen, query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories, 
     word_idx, story_maxlen, query_maxlen)

inputs_train = stack_inputs(inputs_train)
inputs_test = stack_inputs(inputs_test)

inps = [inputs_train, queries_train]
val_inps = [inputs_test, queries_test]


C:\Users\nlandy\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [29]:
print('Build model...')


emb_dim = 20
def emb_sent_bow(inp):
    emb = layers.TimeDistributed(Embedding(vocab_size, emb_dim))(inp)
    return layers.Lambda(lambda x: K.sum(x, 2))(emb)
inp_story = layers.Input((story_maxsents, story_maxlen))
emb_story = emb_sent_bow(inp_story)
inp_story.shape, emb_story.shape
inp_q = layers.Input((query_maxlen,))
emb_q = layers.Embedding(vocab_size, emb_dim)(inp_q)
emb_q = layers.Lambda(lambda x: K.sum(x, 1))(emb_q)
emb_q = layers.Reshape((1, emb_dim))(emb_q)
inp_q.shape, emb_q.shape
x = layers.Dot(axes=2)([emb_story, emb_q])
x = layers.Reshape((story_maxsents,))(x)
x = layers.Activation('softmax')(x)
match = layers.Reshape((story_maxsents,1))(x)
match.shape
emb_c = emb_sent_bow(inp_story)
x = layers.Dot(axes=1)([match, emb_c])
response = layers.Reshape((emb_dim,))(x)
res = layers.Dense(vocab_size, activation='softmax')(response)
model = Model([inp_story, inp_q], res)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
K.set_value(model.optimizer.lr, 1e-2)
hist=model.fit(inps, answers_train, nb_epoch=100, batch_size=32,
           validation_data=(val_inps, answers_test))

print(model.summary())

Build model...


C:\Users\nlandy\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1000 samples, validate on 1000 samples
Epoch 1/100
1000/1000 [==============================] - 1s 1ms/step - loss: 2.1363 - acc: 0.1690 - val_loss: 2.0746 - val_acc: 0.1460
Epoch 2/100
1000/1000 [==============================] - 1s 681us/step - loss: 1.8185 - acc: 0.1830 - val_loss: 1.9328 - val_acc: 0.1870
Epoch 3/100
1000/1000 [==============================] - 1s 713us/step - loss: 1.8089 - acc: 0.2040 - val_loss: 1.8334 - val_acc: 0.1760
Epoch 4/100
1000/1000 [==============================] - 1s 698us/step - loss: 1.8071 - acc: 0.1870 - val_loss: 1.8663 - val_acc: 0.1870
Epoch 5/100
1000/1000 [==============================] - 1s 651us/step - loss: 1.7812 - acc: 0.2220 - val_loss: 1.8387 - val_acc: 0.1830
Epoch 6/100
1000/1000 [==============================] - 1s 687us/step - loss: 1.6783 - acc: 0.3270 - val_loss: 1.8265 - val_acc: 0.2240
Epoch 7/100
1000/1000 [==============================] - 1s 807us/step - loss: 1.5635 - acc: 0.3900 - val_loss: 1.7593 - val_acc: 0.

In [ ]:
emb_dim = 30
def emb_sent_bow(inp):
    emb_op = layers.TimeDistributed(Embedding(vocab_size, emb_dim, embeddings_regularizer=regularizers.l2(0.002)))
    emb = emb_op(inp)
    emb = layers.Dropout(0.1)(emb)
    emb = layers.Lambda(lambda x: K.sum(x, 2))(emb)
#     return Elemwise(0, False)(emb), emb_op
    return emb, emb_op
inp_story = layers.Input((story_maxsents, story_maxlen))
inp_q = layers.Input((query_maxlen,))
emb_story, emb_story_op = emb_sent_bow(inp_story)
emb_q = emb_story_op.layer(inp_q)
emb_q = layers.Lambda(lambda x: K.sum(x, 1))(emb_q)
h = layers.Dense(emb_dim, kernel_regularizer=regularizers.l2(0.0000))
def one_hop(u, A):
    C, _ = emb_sent_bow(inp_story)
    x = layers.Reshape((1, emb_dim))(u)
    x = layers.Dot(axes=2)([A, x])
    x = layers.Reshape((story_maxsents,))(x)
    x = layers.Activation('softmax')(x)
    match = layers.Reshape((story_maxsents,1))(x)

    x = layers.Dot(axes=1)([match, C])
    x = layers.Reshape((emb_dim,))(x)
    x = h(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Add()([x, emb_q])
    return x, C
response, emb_story = one_hop(emb_q, emb_story)
response, emb_story = one_hop(response, emb_story)
# response, emb_story = one_hop(response, emb_story)
res = layers.Dense(vocab_size, activation='softmax')(response)
answer = Model([inp_story, inp_q], res)
answer.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
K.set_value(answer.optimizer.lr, 5e-3)
hist=answer.fit(inps, answers_train, nb_epoch=100, batch_size=32,
           validation_data=(val_inps, answers_test))

C:\Users\nlandy\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.9122 - acc: 0.1723 - val_loss: 1.8170 - val_acc: 0.1600
Epoch 2/100
10000/10000 [==============================] - 25s 3ms/step - loss: 1.8125 - acc: 0.1735 - val_loss: 1.8120 - val_acc: 0.1670
Epoch 3/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.8045 - acc: 0.1679 - val_loss: 1.7999 - val_acc: 0.1650
Epoch 4/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.8026 - acc: 0.1681 - val_loss: 1.8054 - val_acc: 0.1600
Epoch 5/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.7999 - acc: 0.1664 - val_loss: 1.8011 - val_acc: 0.1600
Epoch 6/100
10000/10000 [==============================] - 27s 3ms/step - loss: 1.7978 - acc: 0.1750 - val_loss: 1.7979 - val_acc: 0.1870
Epoch 7/100
10000/10000 [==============================] - 28s 3ms/step - loss: 1.7973 - acc: 0.1717 - val_loss: 1.8022 - v

10000/10000 [==============================] - 25s 2ms/step - loss: 1.0284 - acc: 0.6990 - val_loss: 0.8580 - val_acc: 0.7440
Epoch 60/100
10000/10000 [==============================] - 25s 3ms/step - loss: 1.0179 - acc: 0.6999 - val_loss: 0.8751 - val_acc: 0.7550
Epoch 61/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.0197 - acc: 0.6954 - val_loss: 0.8471 - val_acc: 0.7410
Epoch 62/100
10000/10000 [==============================] - 25s 2ms/step - loss: 1.0263 - acc: 0.6921 - val_loss: 0.8759 - val_acc: 0.7500
Epoch 63/100
10000/10000 [==============================] - 25s 3ms/step - loss: 1.0281 - acc: 0.6960 - val_loss: 0.9190 - val_acc: 0.7500
Epoch 64/100
10000/10000 [==============================] - 26s 3ms/step - loss: 1.0245 - acc: 0.6982 - val_loss: 0.8920 - val_acc: 0.7410
Epoch 65/100
 3104/10000 [========>.....................] - ETA: 17s - loss: 1.0036 - acc: 0.7033